## Use Detectron2 to calculate dynamic:static cars

- load images and masks
- run detectron2 on each image
- get matrix for each car instance
- compare matrix to mask by IoU
- if IoU is above 0.7 its dynamic, otherwise static
- add to dictionary

In [1]:
%reload_ext autoreload
%autoreload 2

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, random
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# import dataset class and torch
import torch
from DatasetClassRatioComp import ExtendedKittiMod
from torch.utils.data import DataLoader

In [6]:
class DynamicStaticRatio():
    def __init__(self, dataset):
        self.dataset = dataset
        self.cfg = get_cfg()

    def get_predictor(self):
        # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
        yaml_cityscapes = "Cityscapes/mask_rcnn_R_50_FPN.yaml"
        yaml_coco = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
        yaml_path = yaml_cityscapes

        self.cfg.merge_from_file(model_zoo.get_config_file(yaml_path))
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
        # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
        self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(yaml_path)
        predictor = DefaultPredictor(self.cfg)
        return predictor

    def viz(self, im, outputs):
        # We can use `Visualizer` to draw the predictions on the image.
        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        plt.figure(figsize=(15,7))
        plt.imshow(out.get_image()[:, :, ::-1])
        plt.show()

    def calc_iou_moving(self, outputs: torch.Tensor, labels: torch.Tensor):
        SMOOTH = 1e-6
        # outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
        # labels = labels.squeeze(1)
        
        intersection = torch.sum(torch.bitwise_and(outputs, labels).float(), (1,2))  # Will be zero if Truth=0 or Prediction=0
        union = torch.sum(torch.bitwise_or(outputs, labels).float(), (1,2))          # Will be zero if both are 0
        
        IoU = (intersection + SMOOTH) / (union + SMOOTH)   # avoid division by zero
        IoU = IoU.mean()
            
        return IoU

    def get_ratio(self):
        predictor = self.get_predictor()
        static = []
        dynamic = []
        for im, mask in self.dataset:
            outputs = predictor(im)
            # viz(im, outputs)
            # plt.imshow(mask[0].numpy())
            pred_masks = outputs["instances"].pred_masks
            pred_classes = outputs["instances"].pred_classes

            cur_static = 0
            cur_dynamic = 0
            for i in range(pred_masks.shape[0]):
                # check if predicted class == Car
                if pred_classes[i] == 2:
                    iou = self.calc_iou_moving(pred_masks[i,:,:].cpu().int(), mask.int())
                    if iou >= 0.9:
                        cur_dynamic += 1
                    else:
                        cur_static += 1

            static.append(cur_static)
            dynamic.append(cur_dynamic)
        return sum(dynamic)/sum(static)


# loading dataset and configs for NN model
data_root = "/storage/remote/atcremers40/motion_seg/datasets/Extended_MOD_Masks/"

# get train and val dataset
dataset = ExtendedKittiMod(data_root, test=False)
train_size = int(0.8 *  len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size], generator = torch.Generator().manual_seed(0))

# get test dataset
test_set = ExtendedKittiMod(data_root, test=True)

dirs loaded:
['2011_09_26_drive_0014_sync', '2011_09_26_drive_0015_sync', '2011_09_26_drive_0018_sync', '2011_09_26_drive_0032_sync', '2011_09_26_drive_0056_sync', '2011_09_26_drive_0057_sync', '2011_09_26_drive_0059_sync', '2011_09_26_drive_0060_sync', '2011_09_26_drive_0084_sync']
dirs loaded:
['2011_09_26_drive_0005_sync', '2011_09_26_drive_0013_sync', '2011_09_26_drive_0051_sync']


In [7]:
# calc the dynamic:static ratio
ratio_test = DynamicStaticRatio(test_set)
print(f"test set ratio (dynamic:static): {ratio_test.get_ratio()}")

ratio_val = DynamicStaticRatio(val_set)
print(f"val set ratio (dynamic:static): {ratio_val.get_ratio()}")

test set ratio (dynamic:static): 0.01055833929849678
val set ratio (dynamic:static): 0.007982969664715274
